## [Day 2](https://adventofcode.com/2020/day/2)

Alright so this problem gives us a series of codes with rules for determining if they're valid or not. The first two numbers indicate a number of ocurrances that a letter in the code can take. Then a specific letter is given followed by the code. We're asked to count up the number of valid codes.

In [74]:
import numpy as np
import pandas as pd
codes_raw = np.genfromtxt('../inputs/d2.txt', dtype = 'U')
codes = pd.DataFrame(codes_raw, columns = ['range', 'letter', 'code'])
codes.head()

,range,letter,code
0,1-5,k:,kkkkhkkkkkkkkkk
1,5-7,k:,blkqhtxfgktdkxzkksk
2,15-16,x:,xxxxxxxxxxxxxxlf
3,3-5,j:,fjpvj
4,17-20,x:,zsxjrxkgxxxxxxxmxgxf


Alright, this imported in a pretty helpful way. Now we wanna split it up

In [75]:
# Now we got a little more cleanup to do.
range_splits = codes.range.str.split(pat = '-', n = 2, expand=True)
range_splits.columns = ['lower', 'upper']
range_splits = range_splits.astype('i')
codes = pd.concat([codes, range_splits], axis = 1)
# That was pretty easy with the expand option

codes.letter = codes.letter.str.rstrip(':')
codes.head()

,range,letter,code,lower,upper
0,1-5,k,kkkkhkkkkkkkkkk,1,5
1,5-7,k,blkqhtxfgktdkxzkksk,5,7
2,15-16,x,xxxxxxxxxxxxxxlf,15,16
3,3-5,j,fjpvj,3,5
4,17-20,x,zsxjrxkgxxxxxxxmxgxf,17,20


In [77]:
# Now we wanna compute the count for each letter within the code
def str_count(letter, word):
    return word.count(letter)
codes['letter_count'] = codes.apply(lambda x: str_count(letter = x['letter'], word = x['code']), axis = 1) 
codes.head()

,range,letter,code,lower,upper,letter_count
0,1-5,k,kkkkhkkkkkkkkkk,1,5,14
1,5-7,k,blkqhtxfgktdkxzkksk,5,7,6
2,15-16,x,xxxxxxxxxxxxxxlf,15,16,14
3,3-5,j,fjpvj,3,5,2
4,17-20,x,zsxjrxkgxxxxxxxmxgxf,17,20,11


So just a little aside on the `apply`. This is a bit counterintuitive since the vectorized functions in R make this easy. What we're doing is passing an anonymous function to `apply` and the argument is the row of the data frame. Then we compute the value from `str_count`. 

In [78]:
len(codes.query('lower <= letter_count <= upper'))

519

### Part 2



Now we have a spin on the question where we need to find which codes have exactly one matching letter in the positions defined by `lower` and `upper`. Here they mean 1 to be the first letter, not index.

In [84]:
# I guess we'll try to solve it in a similar way:
def check_code(code, letter, left, right):
    left = code[left-1]
    right = code[right-1]
    return (left == letter) ^ (right == letter)

codes['letter_match'] = codes.apply(lambda x: check_code(
    code = x['code'], 
    letter = x['letter'], 
    left = x['lower'],
    right = x['upper']
), axis = 1) 
codes.head()

,range,letter,code,lower,upper,letter_count,letter_match
0,1-5,k,kkkkhkkkkkkkkkk,1,5,14,True
1,5-7,k,blkqhtxfgktdkxzkksk,5,7,6,False
2,15-16,x,xxxxxxxxxxxxxxlf,15,16,14,False
3,3-5,j,fjpvj,3,5,2,True
4,17-20,x,zsxjrxkgxxxxxxxmxgxf,17,20,11,True


In [85]:
len(codes.query('letter_match'))

708